# Process OGC Web Feature Service

This notebook uses an endpoint URL, passed either as an 'endpoint' parameter when the notebook is called, or assigned in cell 5.



In [1]:
import sys
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')
    
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests

#from owslib.wfs import WebFeatureService

SystemExit: ERROR: cannot find GDAL/OGR modules

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
#import xmltodict

In [8]:
%%javascript
//check if there is an endpoint parameter in the notebook call

function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("endpoint='".concat(getQueryStringValue("endpoint")).concat("'"));
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [9]:
url_partitioned = full_notebook_url.partition('ISOmetadata-ExtractDistributions.ipynb')
base_url = url_partitioned[0];

if(endpoint is not None and len(endpoint)>0):
    print('Got endpoint parameter:',endpoint)
else: 
    # some USGS ScienceBase Geoserver WFS endpoints:
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c54be4b0aa151574a8dc'
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c5fce4b0aa151574a8ed'
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/4f4e49cfe4b07f02db5da90e'

    # some NGDS end points
    endpoint='http://services.azgs.az.gov/arcgis/services/aasggeothermal/CAaqWellChemistry/MapServer/WFSServer'
    #endpoint='http://services.azgs.az.gov/arcgis/services/aasggeothermal/AZActiveFaults/MapServer/WFSServer'

    #EarthChem
    #endpoint='http://ecp.iedadata.org/wfs/ows?service=wfs'

    #smu thermal conductivity, geoserver
    #endpoint='http://geothermal.smu.edu:9000/geoserver/aasg-thermalconductivity/ows'

    #NOAA Watch/warning polygons
    #endpoint = 'https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Forecasts_Guidance_Warnings/watch_warn_adv/MapServer/WFSServer'
    print('Assign endpoint:', endpoint)
# limit the number of features that will be processed
maxfeatures=500000

Got endpoint parameter: https://www.sciencebase.gov/catalogMaps/mapping/ows/5032ab9de4b0d64661a77224


In [10]:
## Shapefile available?
driverName = "WFS"
# Set the driver (optional)
wfs_drv = ogr.GetDriverByName(driverName)

if wfs_drv is None:
    print ("%s driver not available.\n" % driverName)
else:
    print  ("%s driver IS available.\n" % driverName)

NameError: name 'ogr' is not defined

In [114]:
# !set GDAL_DATA=E:\EPrograms\Anaconda\Library\share\gdal
# !set GDAL_DATA

In [115]:
# Speeds up querying WFS capabilities for services with alot of layers
gdal.SetConfigOption('OGR_WFS_LOAD_MULTIPLE_LAYER_DEFN', 'NO')

# Set config for paging. Works on WFS 2.0 services and WFS 1.0 and 1.1 with some other services.
gdal.SetConfigOption('OGR_WFS_PAGING_ALLOWED', 'YES')
gdal.SetConfigOption('OGR_WFS_PAGE_SIZE', '10000')


#test that service is up with getcapabilities request

capurl=endpoint + '?service=wfs&request=getCapabilities'
r = requests.head(capurl)
if (r.status_code != requests.codes.ok):
    print('ERROR: service not responding')
else:
    # Open the webservice
    wfs_ds = wfs_drv.Open('WFS:' + endpoint)
    #wfs_ds = wfs_drv.Open('http://ecp.iedadata.org/wfs/ows?service=wfs')
    if wfs_ds is None:
        print('ERROR: can not open WFS datasource')


In [116]:
# iterate over available layers
for i in range(wfs_ds.GetLayerCount()):
    layer = wfs_ds.GetLayerByIndex(i)
    srs = layer.GetSpatialRef()
    print ('Layer: %s, Features: %s, SR: %s...' % (layer.GetName(), layer.GetFeatureCount(), srs.ExportToWkt()[0:50]))

    # iterate over features
    #fcount = 1
    #feat = layer.GetNextFeature()
    #while feat is not None:
    #    feat = layer.GetNextFeature()
    #    fcount = fcount + 1
        # do something more..
    #feat = None
    print ('feature count: ',layer.GetFeatureCount())

Layer: aasg:RadiogenicHeatProduction, Features: 2319, SR: GEOGCS["NAD27",DATUM["North_American_Datum_1927",S...
('feature count: ', 2319)


#gdal call to get the capabilities document; change to type 'code' cell and import xmltodict to run this
caps=wfs_ds.GetLayerByName('WFSGetCapabilities')
#print the number of feature types reported in the capabilities document
print(caps.GetFeatureCount())

for feat in caps:
    jsonwrap= json.loads(feat.ExportToJson())
    
capsxml=jsonwrap['properties']['content']
capsjson=xmltodict.parse(capsxml)
capsjson=json.dumps(capsjson, indent=2)
#json dump of the capabilities document
#print(capsjson)

Want to let user pick the layer to work with.

In [118]:

feat_list={}
for i in range(wfs_ds.GetLayerCount()):
    layer = wfs_ds.GetLayerByIndex(i)
    feat_list[layer.GetName()] = layer.GetName()


def f(feat_menu):
    return feat_menu
# out = interact(f, feat_menu=nb_menu);
out = interact(f, feat_menu=feat_list.keys());

#print("interact out: ", out)

interactive(children=(Dropdown(description=u'feat_menu', options=('aasg:RadiogenicHeatProduction',), value='aaâ€¦

In [119]:
# Get a specific layer

chosenlayer= feat_list[out.widget.result]
if len(chosenlayer)==0:
    chosenlayer=wfs_ds.GetLayerByIndex(1).GetName()

layer = wfs_ds.GetLayerByName(chosenlayer)

print('chosen layer:',chosenlayer)

if not layer:
    sys.exit('ERROR: can not find layer in service')
else:
    pass

('chosen layer:', 'aasg:RadiogenicHeatProduction')


In [120]:
%%time
count = 0
tdata = []
#feat = layer.GetNextFeature()
#print (json.loads(feat.ExportToJson()))
#while count < 10:
#while feat is not None:

#print(layer.GetFeatureCount())
#feat=layer.GetNextFeature()
#print(feat)

if (layer.GetFeatureCount()==1):
    feat=layer.GetFeature(1)
    jtemp=feat.ExportToJson()
    print('J temp:',jtemp)
    tdata.append(json.loads(jtemp))   
else:
    feat = layer.GetNextFeature()
    while feat is not None:
    #for feat in layer:
        #print (json.loads(feat.ExportToJson()))
        tdata.append(json.loads(feat.ExportToJson()))
        feat = layer.GetNextFeature()
        count = count + 1
        if count > maxfeatures:
            break
feat = None

#print (tdata)
dataframe=json_normalize(tdata)
print('size:',dataframe.size)

#dataframe
print('done timed cell,',count,'features in result set')

('size:', 51000)
('done timed cell,', 1000, 'features in result set')
Wall time: 1.97 s


Problem with json_normalize:  
This appears to be a bug in the latest version of pandas: https://github.com/pandas-dev/pandas/issues/21158
Running pandas '0.23.0', error arises due to condition case when null value occurs on the nesting level greater than 0. It seems to have been changed around two months ago that seems to have made it's way into 0.23.0 release two weeks ago:

https://github.com/pandas-dev/pandas/commit/01882ba5b4c21b0caf2e6b9279fb01967aa5d650#diff-9c654764f5f21c8e9d58d9ebf14de86d

When I use python verion 3.6.3 :: Anaconda Inc. and pandas version 0.20.3 I do not see this issue and json_normalize is able to work properly. 

2018-06-28 SMR rolled pandas back to 0.20.3 in the py36 kernel env.


Now get a list of the attributes of the feature type

Do some data summarization with pandas

In [121]:
#dataframe.dtypes

In [122]:
#for value in dataframe['properties.K_mgL']:
#    if value != None:
#        print (value)

In [123]:
print('%s features analyzed.' % len(dataframe.index))

for col in dataframe.columns:
    #print(col)
    if (dataframe[col].count() > 0) and  (dataframe[col].dtype=='float64'):
        mean=dataframe[col].mean()
        max=dataframe[col].max()
        min=dataframe[col].min()
        print('field:%s, count:%s, mean:%s; max:%s; min:%s' 
              % (str(col.split('.')[1]), dataframe[col].count(), "{:.2f}".format(mean), 
                 "{:.2f}".format(max), "{:.2f}".format(min)))
    
    if (dataframe[col].count() > 0) and  (type(dataframe[col][1]) is str):
        # and  (type(dataframe[col][1])=='str')
        #for item in dataframe[col].unique():
        #    print (item)
        #print (type(dataframe[col][1]))
        if len(dataframe.index)==len(dataframe[col].unique()):
            print(col,'is text field, all values unique')
            print('  ')
        elif (len(dataframe[col].unique()) > 50):
            print(col,'appears to be a text field, there are',len(dataframe[col].unique()),'unique values.' )
            print('  ')
        else:
            print (col,'has', len(dataframe[col].unique()), 'unique values.')
            print(dataframe[col].unique())
            print('  ')
 
emptyfields = []
print('\nThe following fields contain no data:')
for col in dataframe.columns:            
    if (dataframe[col].count() == 0):
        emptyfields.append(str(col.split('.')[1]))       
print(emptyfields)
        

1000 features analyzed.
field:HeatProduction_mWm3, count:1000, mean:2.05; max:78.00; min:0.01
field:LatDegree, count:1000, mean:51.99; max:82.31; min:43.45
field:LongDegree, count:1000, mean:-102.26; max:-51.20; min:-133.45
field:Potassium, count:1000, mean:2.43; max:7.38; min:0.00
field:SampleDepth_m, count:1000, mean:240.53; max:5700.00; min:0.00
field:SampleMass_kg, count:1000, mean:0.33; max:0.33; min:0.33
field:Thorium, count:1000, mean:10.95; max:119.00; min:0.00
field:Uranium, count:1000, mean:3.83; max:73.60; min:0.00

The following fields contain no data:
['ElevationDatum_m', 'FeatureOfInterestURI', 'Laboratory', 'LaboratoryURI', 'MetadataURI', 'Notes', 'OverburdenThickness_m', 'PLSS_Meridians', 'Range', 'SampleCollectionDate', 'SampleCureDuration', 'SampleDensity_g-cc', 'SectionPart', 'Section_', 'TectonicProvince', 'UncertaintyDensity']


In [124]:
 dataframe.describe()


,id,properties.HeatProduction_mWm3,properties.LatDegree,properties.LongDegree,properties.OBJECTID,properties.Potassium,properties.SampleDepth_m,properties.SampleMass_kg,properties.Thorium,properties.Uranium
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000
mean,500.500000,2.051910,51.988359,-102.258600,500.500000,2.432120,240.525650,3.300000e-01,10.948860,3.831630
std,288.819436,3.193779,7.059343,21.635126,288.819436,1.521545,354.043679,6.664671e-16,12.735285,5.205746
min,1.000000,0.010000,43.454000,-133.447000,1.000000,0.000000,0.000000,3.300000e-01,0.000000,0.000000
25%,250.750000,0.590000,49.110000,-121.555000,250.750000,1.097500,10.750000,3.300000e-01,2.330000,0.960000
50%,500.500000,1.230000,49.900000,-102.420000,500.500000,2.250000,159.100000,3.300000e-01,6.330000,1.960000
75%,750.250000,2.520000,52.829750,-85.486000,750.250000,3.730000,340.125000,3.300000e-01,14.962500,4.432500
max,1000.000000,78.000000,82.307000,-51.200000,1000.000000,7.380000,5700.000000,3.300000e-01,119.000000,73.600000
